# 한국어 텍스트 감정 분석 - PyTorch 모델 학습

이 노트북은 전처리된 데이터를 사용하여 PyTorch 기반의 BERT/RoBERTa 모델을 학습합니다.

## 주요 기능
- PyTorch 네이티브 학습 루프 (Transformers Trainer 대신)
- LoRA 파인튜닝
- WandB 실험 추적
- 혼합 정밀도 학습 (Mixed Precision)
- 조기 종료 및 모델 체크포인팅
- 테스트 데이터 추론 및 제출 파일 생성


In [1]:
# Library Import
import os
import math
import warnings
from collections import Counter
import platform
import sys

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import LinearLR
from transformers import get_linear_schedule_with_warmup
from torch.cuda.amp import autocast, GradScaler
from tqdm.auto import tqdm
import wandb

# Transformers
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    set_seed,
    AutoModelForMaskedLM
)

# PEFT for LoRA
from peft import get_peft_model, LoraConfig, TaskType

# Sklearn
from sklearn.metrics import accuracy_score, f1_score, classification_report

# 경고 메시지 필터링
warnings.filterwarnings("ignore")

print("✅ 라이브러리 임포트 완료")


/data/ephemeral/home/py310/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/data/ephemeral/home/py310/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotat

✅ 라이브러리 임포트 완료


In [2]:
EXPERIMENT_NAME = "DAPT"
PROJECT_NAME = f"[domain_project]_Experiment_{EXPERIMENT_NAME}"
MODEL_NAME = "kykim_bert-kor-base"

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
# 환경 설정
RANDOM_STATE = 42
set_seed(RANDOM_STATE)

# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"디바이스: {device}")

if torch.cuda.is_available():
    print(f"GPU 개수: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"   GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("⚠️  CUDA 사용 불가 - CPU로 훈련 진행")



디바이스: cuda
GPU 개수: 1
   GPU 0: Tesla V100-SXM2-32GB


In [4]:
from dotenv import load_dotenv
import os

# 이 함수가 .env 파일을 읽어서 환경 변수로 로드합니다.
load_dotenv()


True

In [5]:
molels = ["beomi_kcbert-base", "klue_roberta-base", "klue_bert-base","kykim_bert-base", "monologg_koelectra-base-v3-discriminator"]

In [ ]:
# 하이퍼파라미터 설정

class Config:
    # 모델 설정
    model_name = MODEL_NAME
    base_models_path = "/data/ephemeral/home/code/basemodels/"
    local_model_path = base_models_path + MODEL_NAME
    num_classes = 4
    max_length = 128
    
    # 훈련 설정
    unsupervised_batch_size = 128
    batch_size = 256
    eval_batch_size = 256
    num_mlm_epochs = 20
    unsupervised_num_epochs = 20
    num_epochs = 20
    learning_rate = 2e-5
    weight_decay = 0.01
    warmup_steps = 500
    
    # 기타 설정
    gradient_accumulation_steps = 1
    max_grad_norm = 1.0
    early_stopping_patience = 5
    save_best_model = True
    
    # WandB 설정
    use_wandb = True
    project_name = PROJECT_NAME
    run_name = f"{EXPERIMENT_NAME}-training"

config = Config()
print("✅ 하이퍼파라미터 설정 완료")


✅ 하이퍼파라미터 설정 완료


In [7]:
# 데이터 로드
print("전처리된 데이터 로드 중...")

# 훈련 데이터 로드
train_df = pd.read_csv("data/train_processed.csv")
print(f"훈련 데이터: {len(train_df):,}개")

# 검증 데이터 로드
val_df = pd.read_csv("data/val_processed.csv")
print(f"검증 데이터: {len(val_df):,}개")

test_df = pd.read_csv("data/test_processed.csv")
print(f"테스트 데이터: {len(test_df):,}개")

# 라벨 매핑
LABEL_MAPPING = {0: "강한 부정", 1: "약한 부정", 2: "약한 긍정", 3: "강한 긍정"}

print("✅ 데이터 로드 완료")


전처리된 데이터 로드 중...


훈련 데이터: 221,016개
검증 데이터: 55,255개
테스트 데이터: 59,928개
✅ 데이터 로드 완료


In [8]:
# 데이터셋 클래스 정의
class ReviewDataset(Dataset):
    """
    리뷰 텍스트 데이셋 클래스
    - BERT 모델 훈련/추론을 위한 PyTorch Dataset 구현
    - 텍스트 토크나이징 및 텐서 변환 처리
    """

    def __init__(self, texts, labels, tokenizer, max_length):
        """
        데이터셋 초기화
        """
        self.texts, self.labels, self.tokenizer, self.max_length = (
            texts,
            labels,
            tokenizer,
            max_length,
        )

    def __len__(self):
        """데이터셋 크기 반환"""
        return len(self.texts)

    def __getitem__(self, idx):
        """
        특정 인덱스의 데이터 아이템 반환
        """
        # 텍스트 토크나이징 및 패딩
        encoding = self.tokenizer(
            str(self.texts.iloc[idx]),
            truncation=True,  # 최대 길이 초과시 자르기
            padding="max_length",  # 최대 길이까지 패딩
            max_length=self.max_length,
            return_tensors="pt",  # PyTorch 텐서로 반환
        )

        # 기본 아이템 구성 (input_ids, attention_mask)
        item = {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
        }

        # labels가 None이 아닌 경우에만 추가 (train/valid용)
        if self.labels is not None:
            item["labels"] = torch.tensor(self.labels.iloc[idx], dtype=torch.long)

        return item

print("✅ 데이터셋 클래스 정의 완료")


✅ 데이터셋 클래스 정의 완료


In [16]:
class UnlabeledDataset(Dataset):
    """
    라벨이 없는 데이터셋 클래스 (MLM 마스킹 지원)
    """
    def __init__(self, texts, tokenizer, max_length, mask_probability=0.15):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.mask_probability = mask_probability
        self.mask_token_id = tokenizer.mask_token_id
        self.vocab_size = tokenizer.vocab_size
    
    def __len__(self):
        return len(self.texts)
    
    def _apply_mlm_masking(self, input_ids, attention_mask):
        """MLM 스타일 마스킹 적용"""
        labels = input_ids.clone()
        
        # 패딩 토큰이 아닌 위치만 선택
        non_padding_mask = attention_mask.bool()
        
        # 마스킹할 토큰 선택 (패딩 토큰 제외)
        candidate_indices = torch.where(non_padding_mask)[0]
        
        if len(candidate_indices) == 0:
            return input_ids, labels
        
        # 마스킹할 토큰 수 계산
        num_tokens_to_mask = max(1, int(len(candidate_indices) * self.mask_probability))
        
        # 랜덤하게 토큰 선택
        selected_indices = torch.randperm(len(candidate_indices))[:num_tokens_to_mask]
        selected_positions = candidate_indices[selected_indices]
        
        # 선택된 토큰에 대해 마스킹 적용
        for pos in selected_positions:
            # 80% 확률로 [MASK] 토큰으로 교체
            if torch.rand(1) < 0.8:
                input_ids[pos] = self.mask_token_id
            # 10% 확률로 랜덤 토큰으로 교체
            elif torch.rand(1) < 0.1:
                input_ids[pos] = torch.randint(0, self.vocab_size, (1,))
            # 10% 확률로 원본 토큰 유지
        
        return input_ids, labels
    
    def __getitem__(self, idx):
        """인덱스에 해당하는 샘플 반환 (MLM 마스킹 적용)"""
        # 텍스트 추출 - 문자열로 변환 보장
        if hasattr(self.texts, 'iloc'):
            text = str(self.texts.iloc[idx])  # pandas Series인 경우
        else:
            text = str(self.texts[idx])  # list나 다른 타입인 경우
        
        # None이나 NaN 값 처리
        if text == 'None' or text == 'nan' or text == '':
            text = "빈 텍스트입니다."  # 기본값 설정
        
        # 텍스트 토크나이징
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        input_ids = encoding['input_ids'].flatten()
        attention_mask = encoding['attention_mask'].flatten()
        
        # MLM 마스킹 적용
        masked_input_ids, labels = self._apply_mlm_masking(input_ids, attention_mask)
        
        return {
            'input_ids': masked_input_ids,
            'attention_mask': attention_mask,
            'labels': labels  # MLM 학습을 위한 라벨
        }

In [17]:
# 평가 메트릭 함수
def compute_metrics(predictions, labels):
    """
    모델 평가 메트릭 계산 함수
    """
    # 예측값에서 가장 높은 확률의 클래스 선택
    predictions = np.argmax(predictions, axis=1)
    
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average="weighted")
    
    return {
        "accuracy": accuracy,
        "f1": f1,
    }

print("✅ 평가 메트릭 함수 정의 완료")


✅ 평가 메트릭 함수 정의 완료


In [18]:
def model_load_from_local(local_model_path : str = None, model_name : str = None):
    # 1. 토크나이저 로드
    # 절대 경로를 사용하여 로컬 모델 로드
    print("로컬 경로에서 토크나이저 로딩 중...")

    # 경로가 존재하는지 확인
    if not os.path.exists(local_model_path):
        print(f"❌ 경로가 존재하지 않습니다: {local_model_path}")
        return None, None
    else:
        print(f"✅ 경로 확인됨: {local_model_path}")

    tokenizer = AutoTokenizer.from_pretrained(local_model_path)

    # 2. 모델 로드
    # 마찬가지로 로컬 경로(local_model_path)를 사용
    print("로컬 경로에서 모델 로딩 중...")
    model = AutoModelForMaskedLM.from_pretrained(
        local_model_path,
        num_labels=7  # 예: KLUE-TC 감성 분석 클래스 개수
    )

    print("✅ 로컬 스냅샷에서 모델과 토크나이저 로딩 성공!")

    # --- 이제 평소처럼 모델을 사용할 수 있습니다 ---
    inputs = tokenizer("이 영화 정말 재미있네요!", return_tensors="pt")
    outputs = model(**inputs)
    print(outputs.logits)

    return model, tokenizer

In [19]:
model, tokenizer = model_load_from_local(config.local_model_path, config.model_name)

로컬 경로에서 토크나이저 로딩 중...
✅ 경로 확인됨: /data/ephemeral/home/code/basemodels/kykim_bert-kor-base
로컬 경로에서 모델 로딩 중...


Some weights of the model checkpoint at /data/ephemeral/home/code/basemodels/kykim_bert-kor-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


✅ 로컬 스냅샷에서 모델과 토크나이저 로딩 성공!
tensor([[[ -8.7732,  -5.3598,  -4.4707,  ...,  -5.5349,  -4.1685,  -6.2984],
         [-13.5281,  -7.3256,  -5.6978,  ...,  -4.6070,  -7.2017, -13.3684],
         [-13.5850, -10.6456,  -6.2533,  ...,  -5.0207,  -6.9485, -10.5358],
         ...,
         [-13.1823,  -8.8666,  -4.9712,  ..., -10.3408,  -7.1187, -11.3603],
         [-11.3309,  -6.7285,  -2.5937,  ...,  -6.8013,  -4.9377, -10.5218],
         [ -9.3873,  -4.1437,  -6.7594,  ...,  -3.4999,  -1.9037,  -9.5742]]],
       grad_fn=<ViewBackward0>)


In [20]:
# LoRA 설정 적용
# print("LoRA 설정 적용 중...")
# peft_config = LoraConfig(
#     task_type=TaskType.SEQ_CLS,
#     r=config.lora_r,
#     lora_alpha=config.lora_alpha,
#     target_modules=["query", "key", "value"],
#     lora_dropout=config.lora_dropout,
#     bias="none",
# )

# model = get_peft_model(model, peft_config)
# model.print_trainable_parameters()

# 모델을 디바이스로 이동
model = model.to(device)

print("✅ 모델 및 토크나이저 로드 완료")


✅ 모델 및 토크나이저 로드 완료


In [21]:
model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(42000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [22]:
# 데이터셋 및 데이터로더 생성
print("데이터셋 및 데이터로더 생성 중...")

unlabeled_df = pd.DataFrame(pd.concat([
    train_df["review"], 
    val_df["review"], 
    test_df["review"]
], ignore_index=True))


# 훈련 데이터셋 생성
train_dataset = ReviewDataset(
    train_df["review"],
    train_df["label"],
    tokenizer,
    config.max_length,
)

# 검증 데이터셋 생성
val_dataset = ReviewDataset(
    val_df["review"],
    val_df["label"],
    tokenizer,
    config.max_length,
)

unlabeled_dataset = UnlabeledDataset(
    unlabeled_df["review"],
    tokenizer,
    config.max_length,
)

# 데이터로더 생성
train_dataloader = DataLoader(
    train_dataset,
    batch_size=config.batch_size,
    shuffle=True,
    num_workers=0,
    pin_memory=True if torch.cuda.is_available() else False
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=config.eval_batch_size,
    shuffle=False,
    num_workers=0,
    pin_memory=True if torch.cuda.is_available() else False
)

unlabeled_dataloader = DataLoader(
    unlabeled_dataset,
    batch_size=config.unsupervised_batch_size,
    shuffle=True,
    num_workers=0,
    pin_memory=True if torch.cuda.is_available() else False
)



print(f"훈련 데이터: {len(train_dataset):,}개")
print(f"검증 데이터: {len(val_dataset):,}개")
print(f"Unlabeled 데이터: {len(unlabeled_dataset):,}개")
print("✅ 데이터셋 및 데이터로더 생성 완료")


데이터셋 및 데이터로더 생성 중...
훈련 데이터: 221,016개
검증 데이터: 55,255개
Unlabeled 데이터: 336,199개
✅ 데이터셋 및 데이터로더 생성 완료


In [35]:
# 옵티마이저 및 스케줄러 설정
print("옵티마이저 및 스케줄러 설정 중...")

# 옵티마이저 설정
optimizer = AdamW(
    model.parameters(),
    lr=config.learning_rate,
    weight_decay=config.weight_decay
)

# 전체 훈련 스텝 계산
total_steps = len(train_dataloader) * config.num_epochs // config.gradient_accumulation_steps

# 스케줄러 설정 (warmup + linear decay)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=config.warmup_steps,
    num_training_steps=total_steps
)

# 혼합 정밀도 스케일러
scaler = GradScaler() if torch.cuda.is_available() else None

print(f"총 훈련 스텝: {total_steps:,}")
print(f"워밍업 스텝: {config.warmup_steps:,}")
print("✅ 옵티마이저 및 스케줄러 설정 완료")


옵티마이저 및 스케줄러 설정 중...
총 훈련 스텝: 864
워밍업 스텝: 500
✅ 옵티마이저 및 스케줄러 설정 완료


In [25]:
# WandB 초기화
if config.use_wandb:
    wandb.init(
        project=config.project_name,
        name=config.run_name,
        config={
            "model_name": config.model_name,
            "num_epochs": config.num_epochs,
            "batch_size": config.batch_size,
            "learning_rate": config.learning_rate,
            "max_length": config.max_length,
            "num_classes": config.num_classes,
            "warmup_steps": config.warmup_steps,
            "weight_decay": config.weight_decay,
            # "lora_r": config.lora_r,
            # "lora_alpha": config.lora_alpha,
            # "lora_dropout": config.lora_dropout,
            "random_seed": RANDOM_STATE
        }
    )
    print("✅ WandB 초기화 완료")
else:
    print("⚠️  WandB 사용 안함")


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


✅ WandB 초기화 완료


In [26]:
def unsupervie_train_epoch(model, dataloader, optimizer, scheduler, scaler, device, config):
    """한 에포크 훈련 (MLM 비지도 학습)"""
    model.train()
    total_loss = 0
    num_batches = 0
    
    progress_bar = tqdm(dataloader, desc="Unsupervised Training")
    
    for step, batch in enumerate(progress_bar):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)  # MLM을 위한 원본 토큰 라벨
        
        # 그래디언트 초기화
        optimizer.zero_grad()
        
        # 혼합 정밀도 학습
        if scaler is not None:
            with autocast():
                # MLM 손실 계산 - labels를 올바르게 처리
                outputs = model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    labels=labels  # labels는 input_ids와 동일한 shape이어야 함
                )
                loss = outputs.loss
                
            # 스케일된 그래디언트 계산
            scaler.scale(loss).backward()
            
            # 그래디언트 클리핑
            if config.max_grad_norm > 0:
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), config.max_grad_norm)
            
            # 옵티마이저 스텝
            scaler.step(optimizer)
            scaler.update()
            
        else:
            # 일반 정밀도 학습
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            loss = outputs.loss
            
            # 역전파
            loss.backward()
            
            # 그래디언트 클리핑
            if config.max_grad_norm > 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), config.max_grad_norm)
            
            # 옵티마이저 스텝
            optimizer.step()
        
        # 스케줄러 스텝
        if scheduler is not None:
            scheduler.step()
        
        # 손실 누적
        total_loss += loss.item()
        num_batches += 1
        
        # 진행률 표시 업데이트
        avg_loss = total_loss / num_batches
        progress_bar.set_postfix({
            'loss': f'{avg_loss:.4f}',
            'lr': f'{optimizer.param_groups[0]["lr"]:.2e}'
        })
        
        # 그래디언트 누적 (선택사항)
        if config.gradient_accumulation_steps > 1:
            if (step + 1) % config.gradient_accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()
                if scheduler is not None:
                    scheduler.step()
    
    # 평균 손실 계산
    avg_loss = total_loss / num_batches if num_batches > 0 else 0
    
    return avg_loss

In [38]:
def train_epoch(model, dataloader, optimizer, scheduler, scaler, device, config):
    """한 에포크 훈련"""
    model.train()
    total_loss = 0
    predictions = []
    labels = []
    
    progress_bar = tqdm(dataloader, desc="Training")
    
    for step, batch in enumerate(progress_bar):
        # 배치를 디바이스로 이동
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        batch_labels = batch["labels"].to(device)
        
        # 그래디언트 초기화
        optimizer.zero_grad()
        
        # 혼합 정밀도 훈련
        if scaler is not None:
            with autocast():
                outputs = model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    labels=batch_labels
                )
                loss = outputs.loss
                logits = outputs.logits
            
            # 스케일된 그래디언트 계산
            scaler.scale(loss).backward()
            
            # 그래디언트 클리핑 (조건부)
            if config.max_grad_norm > 0:
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), config.max_grad_norm)
            
            # 옵티마이저 스텝
            scaler.step(optimizer)
            scaler.update()
            
        else:
            # 일반 정밀도 학습
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=batch_labels
            )
            loss = outputs.loss
            logits = outputs.logits
            
            # 역전파
            loss.backward()
            
            # 그래디언트 클리핑
            if config.max_grad_norm > 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), config.max_grad_norm)
            
            # 옵티마이저 스텝
            optimizer.step()
        
        # 스케줄러 스텝
        if scheduler is not None:
            scheduler.step()
        
        # 손실 누적
        total_loss += loss.item()
        
        # 예측값 저장
        predictions.extend(logits.detach().cpu().numpy())
        labels.extend(batch_labels.detach().cpu().numpy())
        
        # 진행률 업데이트
        progress_bar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'lr': f'{scheduler.get_last_lr()[0]:.2e}'
        })
    
    # 메트릭 계산
    metrics = compute_metrics(predictions, labels)
    metrics['loss'] = total_loss / len(dataloader)
    
    return metrics

In [28]:
# 검증 함수
def validate_epoch(model, dataloader, device):
    """한 에포크 검증"""
    model.eval()
    total_loss = 0
    predictions = []
    labels = []
    
    progress_bar = tqdm(dataloader, desc="Validation")
    
    with torch.no_grad():
        for batch in progress_bar:
            # 배치를 디바이스로 이동
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            batch_labels = batch["labels"].to(device)
            
            # 순전파
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=batch_labels
            )
            
            loss = outputs.loss
            logits = outputs.logits
            
            total_loss += loss.item()
            
            # 예측값 저장
            predictions.extend(logits.detach().cpu().numpy())
            labels.extend(batch_labels.detach().cpu().numpy())
            
            # 진행률 업데이트
            progress_bar.set_postfix({'loss': f'{loss.item():.4f}'})
    
    # 메트릭 계산
    metrics = compute_metrics(predictions, labels)
    metrics['loss'] = total_loss / len(dataloader)
    
    return metrics

print("✅ 검증 함수 정의 완료")


✅ 검증 함수 정의 완료


In [29]:
def unsupervised_training_loop(model, unlabeled_dataloader, optimizer, scheduler, scaler, device, config):
    """비지도 학습 메인 루프"""
    
    # 초기화
    best_unsup_loss = float('inf')
    patience_counter = 0
    
    print("🚀 비지도 학습 시작...")
    print(f"총 에포크: {config.num_epochs}")
    print(f"조기 종료 인내심: {config.early_stopping_patience}")
    
    for epoch in range(config.unsupervised_num_epochs):
        print(f"\n=== Epoch {epoch+1}/{config.unsupervised_num_epochs} ===")
        
        # 비지도 학습 에포크
        unsup_loss = unsupervie_train_epoch(
            model=model,
            dataloader=unlabeled_dataloader,
            optimizer=optimizer,
            scheduler=scheduler,
            scaler=scaler,
            device=device,
            config=config
        )
        
        print(f"MLM Loss: {unsup_loss:.4f}")
        
        # 모델 저장 조건 (손실이 개선되었을 때)
        if config.save_best_model and unsup_loss < best_unsup_loss:
            best_unsup_loss = unsup_loss
            patience_counter = 0
            
            # 모델 저장
            os.makedirs("./best_unsupervised_model", exist_ok=True)
            model.save_pretrained("./best_unsupervised_model")
            tokenizer.save_pretrained("./best_unsupervised_model")
            print(f"✅ 최고 성능 비지도 모델 저장 (MLM Loss: {best_unsup_loss:.4f})")
            
        else:
            patience_counter += 1
            print(f"⏳ 조기 종료 카운터: {patience_counter}/{config.early_stopping_patience}")
        
        # WandB 로깅 (선택사항)
        if config.use_wandb:
            wandb.log({
                "epoch": epoch + 1,
                "unsupervised_loss": unsup_loss,
                "best_unsupervised_loss": best_unsup_loss,
                "learning_rate": optimizer.param_groups[0]['lr']
            })
        
        # 조기 종료 체크
        if patience_counter >= config.early_stopping_patience:
            print(f"🛑 조기 종료: {config.early_stopping_patience} 에포크 동안 개선 없음")
            break
    
    print(f"\n🎯 비지도 학습 완료!")
    print(f"최고 MLM Loss: {best_unsup_loss:.4f}")
    
    return best_unsup_loss

In [22]:
best_loss = unsupervised_training_loop(
    model=model, 
    unlabeled_dataloader=unlabeled_dataloader,
    optimizer=optimizer,
    scheduler=scheduler,
    scaler=scaler,
    device=device,
    config=config
)

🚀 비지도 학습 시작...
총 에포크: 1
조기 종료 인내심: 5

=== Epoch 1/1 ===


Unsupervised Training:   0%|          | 0/2627 [00:00<?, ?it/s]

MLM Loss: 0.2915
✅ 최고 성능 비지도 모델 저장 (MLM Loss: 0.2915)

🎯 비지도 학습 완료!
최고 MLM Loss: 0.2915


In [39]:
# 메인 훈련 루프 (비지도 사전 훈련 모델 기반)
print("=" * 50)
print("비지도 사전 훈련 모델 기반 지도 학습 시작")
print("=" * 50)

# 비지도 사전 훈련 모델 로드
print("🔄 비지도 사전 훈련 모델 로드 중...")
if os.path.exists("./best_unsupervised_model"):
    print("✅ 비지도 사전 훈련 모델 발견, 로드 중...")
    model = AutoModelForSequenceClassification.from_pretrained(
        "./best_unsupervised_model",
        num_labels=config.num_classes
    )
    tokenizer = AutoTokenizer.from_pretrained("./best_unsupervised_model")
    print("✅ 비지도 사전 훈련 모델 로드 완료")
else:
    print("⚠️  비지도 사전 훈련 모델이 없습니다. 기본 모델을 사용합니다.")
    # 기본 모델 로드 (기존 코드)
    model = AutoModelForSequenceClassification.from_pretrained(
        config.local_model_path,
        num_labels=config.num_classes
    )
    tokenizer = AutoTokenizer.from_pretrained(config.local_model_path)

# 모델을 디바이스로 이동
model = model.to(device)

# 훈련 정보 출력
print(f"훈련 샘플: {len(train_dataset):,}개")
print(f"검증 샘플: {len(val_dataset):,}개")
print(f"훈련 에포크: {config.num_epochs}회")
print(f"배치 크기: {config.batch_size} (훈련) / {config.eval_batch_size} (검증)")
print(f"학습률: {config.learning_rate}")
print(f"시드값: {RANDOM_STATE}")
print(f"디바이스: {device}")
print(f"WandB 사용: {config.use_wandb}")

# 조기 종료 및 체크포인팅 설정
best_val_accuracy = 0
patience_counter = 0
training_history = []

# 훈련 시작
try:
    for epoch in range(config.num_epochs):
        print(f"\n{'='*20} Epoch {epoch+1}/{config.num_epochs} {'='*20}")
        
        # 훈련
        train_metrics = train_epoch(
            model, train_dataloader, optimizer, scheduler, scaler, device, config
        )
        
        # 검증
        val_metrics = validate_epoch(model, val_dataloader, device)
        
        # 결과 출력
        print(f"\n훈련 결과:")
        print(f"  Loss: {train_metrics['loss']:.4f}")
        print(f"  Accuracy: {train_metrics['accuracy']:.4f}")
        print(f"  F1: {train_metrics['f1']:.4f}")
        
        print(f"\n검증 결과:")
        print(f"  Loss: {val_metrics['loss']:.4f}")
        print(f"  Accuracy: {val_metrics['accuracy']:.4f}")
        print(f"  F1: {val_metrics['f1']:.4f}")
        
        # WandB 로깅
        if config.use_wandb:
            wandb.log({
                "epoch": epoch + 1,
                "train_loss": train_metrics['loss'],
                "train_accuracy": train_metrics['accuracy'],
                "train_f1": train_metrics['f1'],
                "val_loss": val_metrics['loss'],
                "val_accuracy": val_metrics['accuracy'],
                "val_f1": val_metrics['f1'],
                "learning_rate": scheduler.get_last_lr()[0]
            })
        
        # 히스토리 저장
        training_history.append({
            'epoch': epoch + 1,
            'train_loss': train_metrics['loss'],
            'train_accuracy': train_metrics['accuracy'],
            'train_f1': train_metrics['f1'],
            'val_loss': val_metrics['loss'],
            'val_accuracy': val_metrics['accuracy'],
            'val_f1': val_metrics['f1']
        })
        
        # 최고 성능 모델 저장
        if config.save_best_model and val_metrics['accuracy'] > best_val_accuracy:
            best_val_accuracy = val_metrics['accuracy']
            patience_counter = 0
            
            # 모델 저장 (지도 학습 완료 모델)
            os.makedirs("./best_supervised_model", exist_ok=True)
            model.save_pretrained("./best_supervised_model")
            tokenizer.save_pretrained("./best_supervised_model")
            print(f"✅ 최고 성능 지도 학습 모델 저장 (Accuracy: {best_val_accuracy:.4f})")
        else:
            patience_counter += 1
        
        # 조기 종료 체크
        if patience_counter >= config.early_stopping_patience:
            print(f"\n⚠️  조기 종료: {config.early_stopping_patience} 에포크 동안 개선 없음")
            break
    
    print(f"\n✅ 지도 학습 완료!")
    print(f"최고 검증 정확도: {best_val_accuracy:.4f}")
    
except KeyboardInterrupt:
    print("\n⚠️  사용자에 의해 훈련이 중단되었습니다.")
except Exception as e:
    print(f"\n❌ 훈련 중 오류 발생: {str(e)}")
    raise

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./best_unsupervised_model and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


비지도 사전 훈련 모델 기반 지도 학습 시작
🔄 비지도 사전 훈련 모델 로드 중...
✅ 비지도 사전 훈련 모델 발견, 로드 중...
✅ 비지도 사전 훈련 모델 로드 완료
훈련 샘플: 221,016개
검증 샘플: 55,255개
훈련 에포크: 1회
배치 크기: 256 (훈련) / 256 (검증)
학습률: 2e-05
시드값: 42
디바이스: cuda
WandB 사용: True

==================== Epoch 1/1 ====================


Training:   0%|          | 0/864 [00:00<?, ?it/s]


❌ 훈련 중 오류 발생: unscale_() has already been called on this optimizer since the last update().


RuntimeError: unscale_() has already been called on this optimizer since the last update().

In [ ]:
# 테스트 데이터 추론
print("테스트 데이터 추론 시작...")

# 테스트 데이터 로드
test_df = pd.read_csv("data/test.csv")
print(f"테스트 데이터: {len(test_df):,}개")

# 최고 성능 모델 로드
if os.path.exists("./best_model"):
    print("최고 성능 모델 로드 중...")
    inference_model = AutoModelForSequenceClassification.from_pretrained("./best_model")
    inference_tokenizer = AutoTokenizer.from_pretrained("./best_model")
    inference_model = inference_model.to(device)
    inference_model.eval()
    print("✅ 최고 성능 모델 로드 완료")
else:
    print("⚠️  저장된 모델이 없습니다. 현재 모델 사용")
    inference_model = model
    inference_tokenizer = tokenizer


In [ ]:
# 테스트 데이터셋 및 데이터로더 생성
test_dataset = ReviewDataset(
    test_df["review"],
    None,  # 테스트 데이터는 라벨 없음
    inference_tokenizer,
    config.max_length,
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=config.eval_batch_size,
    shuffle=False,
    num_workers=2,
    pin_memory=True if torch.cuda.is_available() else False
)

print("✅ 테스트 데이터셋 준비 완료")


In [ ]:
# 추론 실행
print("추론 실행 중...")
inference_model.eval()
predictions = []

with torch.no_grad():
    for batch in tqdm(test_dataloader, desc="Inference"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        
        outputs = inference_model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        
        logits = outputs.logits
        batch_predictions = torch.argmax(logits, dim=-1)
        predictions.extend(batch_predictions.cpu().numpy())

print(f"✅ 추론 완료: {len(predictions):,}개 예측")


In [ ]:
# 제출 파일 생성
print("제출 파일 생성 중...")

# 예측 결과를 데이터프레임에 추가
test_df["pred"] = predictions

# 클래스별 예측 분포 확인
unique_predictions, counts = np.unique(predictions, return_counts=True)
print("\n클래스별 예측 분포:")
for pred, count in zip(unique_predictions, counts):
    percentage = (count / len(predictions)) * 100
    class_name = LABEL_MAPPING.get(pred, f"클래스 {pred}")
    print(f"   {class_name} ({pred}): {count:,}개 ({percentage:.1f}%)")

# 샘플 제출 파일 로드
sample_submission = pd.read_csv("data/sample_submission.csv")

# ID를 기준으로 병합하여 제출 파일 생성
submission_df = sample_submission[["ID"]].merge(
    test_df[["ID", "pred"]], 
    left_on="ID", 
    right_on="ID", 
    how="left"
)

# 제출 파일 검증
assert len(submission_df) == len(sample_submission), f"길이 불일치: {len(submission_df)} vs {len(sample_submission)}"
assert submission_df["pred"].isin([0, 1, 2, 3]).all(), "모든 예측값은 [0, 1, 2, 3] 범위에 있어야 합니다"
assert not submission_df["pred"].isnull().any(), "예측값에 null 값이 있으면 안됩니다"
assert not submission_df["ID"].isnull().any(), "ID 컬럼에 null 값이 있으면 안됩니다"

print("✅ 제출 파일 검증 통과")

# 제출 파일 저장
submission_path = "./output.csv"
submission_df.to_csv(submission_path, index=False)
print(f"✅ 제출 파일 저장 완료: {submission_path}")

# WandB 종료
if config.use_wandb:
    wandb.finish()
    print("✅ WandB 세션 종료")

print("\n🎉 모든 작업 완료!")


In [ ]:
# 테스트 데이터 추론
print("=" * 50)
print("테스트 데이터 추론")
print("=" * 50)

# 테스트 데이터 로드
test_df = pd.read_csv("data/test.csv")
print(f"테스트 데이터: {len(test_df):,}개")

# 최고 성능 모델 로드
if os.path.exists("./best_model"):
    print("최고 성능 모델 로드 중...")
    inference_model = AutoModelForSequenceClassification.from_pretrained("./best_model")
    inference_tokenizer = AutoTokenizer.from_pretrained("./best_model")
    inference_model = inference_model.to(device)
    inference_model.eval()
    print("✅ 최고 성능 모델 로드 완료")
else:
    print("⚠️  저장된 모델이 없습니다. 현재 모델 사용")
    inference_model = model
    inference_tokenizer = tokenizer

# 테스트 데이터셋 생성
test_dataset = ReviewDataset(
    test_df["review"],
    None,  # 테스트 데이터는 라벨 없음
    inference_tokenizer,
    config.max_length,
)

# 테스트 데이터로더 생성
test_dataloader = DataLoader(
    test_dataset,
    batch_size=config.eval_batch_size,
    shuffle=False,
    num_workers=2,
    pin_memory=True if torch.cuda.is_available() else False
)

print("✅ 테스트 데이터셋 준비 완료")
